# Equal Weighted Like Portfolio - comparisons

This example compares the performance of an equal weighted portfolio relative to several risk-based optimal portfolios with similar risk profile. 

The CVaR, SMCR, MV, GINI, SMGINI, MAD, LSSD and Omega risk-based portfolios are considered. We use the `rtype=InvNrisk` optimization flag (the optimal portfolio with the same risk as the equal weights portfolio).

The results are presented below. We conclude that, at least for the data in this example, there is no clear-cut preference for a portfolio construction (with a superior performance under all market conditions).


In [14]:
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used as a buffer for market data collected from the data provider (in this case _alphavantage_).
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir`. *For more information see the readMkT documentation. *

In [15]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, dstart = sdate, dend = edate, dir=mktdir, force=False) 

Read GLD form ../MkTdata
Read TLT form ../MkTdata
Read XLV form ../MkTdata
Read VGT form ../MkTdata
Read PSJ form ../MkTdata


### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio,
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR`,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR`,
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSSD'` is the mLSSD optimal portfolio with weights `coef_LSSD`,
- in all cases the optimization type is set to `rtype='InvNRisk'` (optimal portfolio with the same risk profile as the equal weighted portfolio)
- the `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` and `P_SMGINI` where it is set to 1.25 years (for computational speed convenience).

In [16]:
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
rtype = 'InvNrisk'
coef_MAD = [1./3.] * 3
coef_LSSD = [1./3.] * 3

models = {'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'mu': 0., 'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': 3.25}},
          'P_SMCR': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': 3.25}},
          'P_MV': {'type': 'Port_MV', 'm_param': {'mu': 0., 'rtype': rtype, 'hlength': 3.25}},
          'P_MAD': {'type': 'Port_MAD', 'm_param': {'mu': 0., 'coef': coef_MAD, 'rtype': rtype, 'hlength': 3.25}},
          'P_GINI': {'type': 'Port_GINI', 'm_param': {'mu': 0., 'rtype': rtype, 'hlength': 1.25}},
          'P_SMGINI': {'type': 'Port_SMGINI', 'm_param': {'mu': 0., 'rtype': rtype, 'hlength': 1.25}},
          'P_Omega': {'type': 'Port_Omega', 'm_param': {'mu': 0., 'rtype': rtype, 'hlength': 3.25}},
          'P_LSSD': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD, 'rtype': rtype, 'hlength': 3.25}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [17]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [18]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [19]:
_ = ps.port_view_all(componly=True, fancy=True)

### Portfolio performances

Note that, at least for the duration of this simulation, the P_Omega portfolio has a similar rate of returns as 
P_N. However, P_Omega has higher maximum drawdown than P_N. P_N has the smallest maximum drawdown among all portfolios. 

In [20]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_N,14.21,-17.44,0.815212,2020-03-18,2020-02-19,2020-04-24
P_SMGINI,12.82,-18.37,0.697927,2016-12-14,2016-07-08,2017-08-31
P_GINI,12.85,-18.76,0.685171,2016-12-14,2016-07-08,2017-09-11
P_MAD,12.74,-19.70,0.646852,2020-03-20,2020-02-19,2020-05-20
P_SMCR,13.06,-20.40,0.639960,2020-03-20,2020-02-19,2020-05-19
P_CVaR,12.75,-20.08,0.634895,2020-03-20,2020-02-19,2020-05-18
P_LSSD,12.54,-20.03,0.626083,2020-03-20,2020-02-19,2020-05-20
P_Omega,14.21,-23.10,0.615171,2018-12-24,2018-09-14,2019-07-10
P_MV,12.22,-19.91,0.613687,2020-03-20,2020-02-19,2020-05-29


### Annual returns

Remarks:

- 2015 (only half year in this simulation) and 2016, P_N is the best performer.
- 2017 P_N is the worst performer. The best performers are P_GINI and P_SMGINI.
- 2018 P_SMGINI has the best performance follow by P_GINI. P_N is the third.
- 2019 The best performers are P_Omega and P_SMCR closely follow by P_CVaR, P_P_LSSD and P_MAD. P_N is in the middle of the pack while P_GINI and P_SMGINI are the worst.
- 2020 The best performers are P_SMGINI and P_GINI follow by P_N.
- 2021 (first half of the year in this simulation) P_SMGINI and P_GINI are the leaders followed by P_MV and P_N.

Conclusion: Depending on the historical period P_N may be an outperformer or an underperformer. Moreover, similar remarks can be made about any other portfolio strategy.

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.


In [21]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_CVaR,P_GINI,P_LSSD,P_MAD,P_MV,P_N,P_Omega,P_SMCR,P_SMGINI
year,,,,,,,,,
2015,-3.79%,-6.11%,-3.75%,-3.09%,-2.74%,-1.71%,-2.93%,-2.91%,-6.39%
2016,4.48%,-5.10%,4.79%,6.87%,6.65%,8.33%,6.45%,4.77%,-4.78%
2017,26.05%,35.46%,26.41%,26.66%,26.97%,22.96%,30.65%,26.39%,35.02%
2018,-0.29%,3.80%,-0.71%,-1.96%,-0.51%,2.65%,-1.38%,-0.58%,4.07%
2019,29.74%,20.29%,29.33%,29.15%,26.86%,25.78%,29.96%,29.96%,19.92%
2020,23.09%,30.88%,21.98%,19.78%,15.98%,25.55%,24.80%,24.09%,31.09%
2021,1.78%,4.98%,1.76%,3.32%,4.36%,4.77%,2.45%,1.25%,4.97%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [22]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_Omega portfolio

Monthly returns: are volatile.

In [23]:
pp['P_Omega'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-4.79%,3.04%,5.68%,6.91%,2.19%,-1.16%
2,nan%,-0.63%,3.91%,0.60%,5.27%,-4.28%,-2.52%
3,nan%,2.63%,0.51%,-0.05%,-1.08%,-8.59%,-3.95%
4,nan%,0.99%,2.44%,2.18%,3.65%,13.97%,2.18%
5,nan%,1.70%,3.42%,5.59%,-4.31%,7.88%,2.07%
6,-2.25%,2.14%,-3.42%,-4.95%,8.07%,0.32%,-1.12%
7,1.86%,4.75%,4.40%,0.77%,3.77%,5.66%,1.36%
8,-7.89%,-0.53%,2.58%,7.97%,-2.43%,0.61%,nan%
9,-3.73%,1.57%,2.45%,-1.00%,-0.11%,-3.75%,nan%


### P_Omega performance

An example of individual portfolio performance inquiry. 

In [24]:
pp['P_Omega'].port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_Omega,14.21,-23.10,0.615171,2018-12-24,2018-09-14,2019-07-10
VGT,23.24,-31.86,0.729357,2020-03-23,2020-02-19,2020-06-09
PSJ,22.12,-30.69,0.720979,2020-03-16,2020-02-19,2020-05-22
XLV,16.74,-28.39,0.589717,2020-03-23,2020-01-22,2020-07-15
TLT,4.92,-21.34,0.230336,2021-03-18,2020-08-04,NaN
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22


### P_Omega drawdowns

Remarks: The first 2 largest drawdowns are similar. However, the 2018 drawdown was caused by the unexpected Feds rate increase while the 2020 event is the Covid-19 pandemic crash. 

In [25]:
pp['P_Omega'].port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-23.10,2018-12-24,2018-09-14,2019-07-10
2,-21.31,2020-03-20,2020-02-19,2020-05-19
3,-16.93,2015-09-28,2015-07-20,2016-07-01
4,-11.67,2021-03-08,2021-02-12,NaN
5,-8.58,2018-02-08,2018-01-26,2018-02-26


### P_Omega weights

Remarks: At present the P_Omega portfolio is dominated by GLD and VGT.

In [26]:
pp['P_Omega'].get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,0.00,0.00,0.00,0.00,100.00
1,2015-09-25,2015-09-24,2012-06-22,0.00,0.00,0.00,0.00,100.00
2,2015-12-28,2015-12-24,2012-09-24,0.00,0.00,4.62,44.54,50.84
3,2016-03-28,2016-03-24,2012-12-24,0.00,0.00,29.45,0.00,70.55
4,2016-06-27,2016-06-24,2013-03-22,0.00,0.00,23.51,41.85,34.64
5,2016-09-27,2016-09-26,2013-06-26,0.00,0.00,29.44,43.89,26.67
6,2016-12-27,2016-12-23,2013-09-23,0.00,0.00,14.38,85.62,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.00,0.00,9.71,90.29,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.00,0.00,10.88,89.12,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.00,29.90,12.20,57.89,0.00
